- https://www.anthropic.com/engineering/built-multi-agent-research-system
    - https://www.youtube.com/watch?v=os5Qxk9tfr0
- https://langchain-ai.github.io/langgraph/how-tos/multi_agent/
    - https://langchain-ai.github.io/langgraph/agents/multi-agent/

In [1]:
from IPython.display import Image

- A multi-agent system consists of multiple agents (LLMs autonomously using tools in a loop) working together.
- Research work involves open-ended problems where it’s very difficult to **predict the required steps in advance**. You can’t hardcode **a fixed path** for exploring complex topics, as the process is inherently dynamic and path-dependent. 
- The essence of search is compression: **distilling insights from a vast corpus**. 
    - Each subagent also provides **separation of concerns**—distinct tools, prompts, and exploration trajectories—which reduces path dependency and enables thorough, independent investigations.
- Our internal evaluations show that multi-agent research systems excel especially for **breadth-first queries** that involve pursuing multiple independent directions simultaneously.
    - We found that a multi-agent system with **Claude Opus 4 as the lead agent** and **Claude Sonnet 4 subagents** outperformed single-agent Claude Opus 4 by 90.2% on our internal research eval.
    - Multi-agent systems work mainly because they help **spend enough tokens** to solve the problem.
        - test-time compute
- 效率与经济

In [2]:
Image(url='https://www.anthropic.com/_next/image?url=https%3A%2F%2Fwww-cdn.anthropic.com%2Fimages%2F4zrzovbb%2Fwebsite%2F1198befc0b33726c45692ac40f764022f4de1bf2-4584x2579.png&w=3840&q=75', width=500)

- “2025年在美国从事AI智能体的所有公司有哪些？列出至少100家，并提供公司名称、网站、产品、业务描述、他们构建的智能体类型以及所属的垂直行业/领域。”
- 这类问题正是博文中所说的“开放式问题”或“广度优先查询”，很难通过一次性的、线性的步骤来解决。它需要动态地探索、发现和整合信息，这正是多智能体系统所擅长的。
- lead agent
    - 任务分解与委派: 它将复杂的总任务分解成多个可以并行处理的子任务，然后创建（spawn）专门的“子智能体”来执行这些子任务。例如，对于寻找100家公司的任务，它可能会创建多个子智能体，每个负责搜索特定行业或特定类型的AI公司。
    - 与Memory交互: 主智能体需要与一个“记忆”模块交互。为了防止在长流程中因上下文窗口（Context Window）限制而丢失关键信息（比如最初制定的计划），主智能体会将它的研究计划保存在Memory中。这样即使后续对话很长，它也能随时取回最初的计划，保持任务的连贯性。
- Subagents
    - 并行搜索 (Search subagent): 多个搜索子智能体可以同时工作，各自探索问题的不同方面。这极大地提高了效率和覆盖范围。博文强调，这种并行化能将复杂查询的研究时间缩短高达90%。
    - 迭代工作 (自循环箭头): 每个搜索子智能体内部会进行迭代。它会先进行一次搜索，然后根据结果评估信息质量，再决定下一步的搜索查询，如此循环，直到找到满意的结果。博文中称之为“interleaved thinking”（交错思考）。
    - 信息压缩与过滤: 子智能体不仅仅是搜索，它们还扮演着“智能过滤器”的角色，将从大量信息中提炼出的最关键内容（condensing the most important tokens）返回给主智能体，而不是返回原始的、未经处理的数据。
    - 引用处理 (Citations subagent): 这是一个专门化的子智能体。在主智能体整合完所有研究结果、形成初步报告后，会将报告和原始资料交给这个引用子智能体。它的任务是精确地为报告中的每一项声明找到并添加来源引用，确保最终报告的严谨性和可信度。

### principles we learned for prompting agents

- Think like your agents
- Teach the orchestrator how to delegate.
- Scale effort to query complexity.
- Tool design and selection are critical.